In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from commons.azure_authentication import AzureAuthHelper
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent
from azure.ai.agents.models import AgentsNamedToolChoice, AgentsNamedToolChoiceType, FunctionName
from azure.search.documents.indexes.models import SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeSourceReference, AzureOpenAIVectorizerParameters, KnowledgeAgentOutputConfiguration, KnowledgeAgentOutputConfigurationModality
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
import requests
from azure.search.documents import SearchIndexingBufferedSender
from dotenv import load_dotenv


: 

In [ ]:
load_dotenv(override=True)

In [ ]:
AZURE_AI_FOUNDRY_PROJECT_ENDPOINT = os.getenv("AZURE_AI_FOUNDRY_PROJECT_ENDPOINT")
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_BASIC_ENDPOINT = os.getenv("AZURE_SEARCH_BASIC_ENDPOINT")
AZURE_SEARCH_BASIC_INDEX_NAME = os.getenv("AZURE_SEARCH_BASIC_INDEX_NAME")
AZURE_SEARCH_BASIC_API_KEY = os.getenv("AZURE_SEARCH_BASIC_API_KEY")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AGENT_NAME = os.getenv("AGENT_NAME", "my-agentic-rag-agent")
AGENT_MODEL = os.getenv("AGENT_MODEL", "gpt-5-nano")
INDEX_NAME = os.getenv("INDEX_NAME", "agentic-rag-index")
KNOWLEDGE_SOURCE_NAME = os.getenv("KNOWLEDGE_SOURCE_NAME")
microsoft_url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
AZURE_SEARCH_API_VERSION = "2025-08-11"


## **CREATE A SEARCH INDEX**

In [ ]:

index = SearchIndex(
    name=INDEX_NAME,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=AZURE_OPENAI_ENDPOINT,
                    deployment_name=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                    model_name=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)


index_client = SearchIndexClient(endpoint=AZURE_SEARCH_BASIC_ENDPOINT, credential=AzureKeyCredential(AZURE_SEARCH_BASIC_API_KEY))
index_client.create_or_update_index(index)
print(f"Index '{INDEX_NAME}' created or updated successfully.")

## **UPLOAD DOCUMENTS TO INDEX**

In [ ]:
documents = requests.get(microsoft_url).json()

with SearchIndexingBufferedSender(
    endpoint=AZURE_SEARCH_BASIC_ENDPOINT, 
    index_name=INDEX_NAME,
    credential=AzureKeyCredential(AZURE_SEARCH_BASIC_API_KEY)
    ) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{INDEX_NAME}' successfully.")

## **CREATING A KNOWLEDGE SOURCE**

In [ ]:

ks = SearchIndexKnowledgeSource(
    name=KNOWLEDGE_SOURCE_NAME,
    description="Knowledge source for Earth at night data",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=INDEX_NAME,
        source_data_select="id,page_chunk,page_number",
    ),
)

index_client = SearchIndexClient(endpoint=AZURE_SEARCH_BASIC_ENDPOINT, credential=AzureKeyCredential(AZURE_SEARCH_BASIC_API_KEY))
index_client.create_or_update_knowledge_source(knowledge_source=ks, api_version=AZURE_SEARCH_API_VERSION)
print(f"Knowledge source '{KNOWLEDGE_SOURCE_NAME}' created or updated successfully.")

## **CREATING A KNOWLEDGE AGENT**

In [ ]:
aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=AZURE_OPENAI_ENDPOINT,
    deployment_name=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    model_name=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
)

output_cfg = KnowledgeAgentOutputConfiguration(
    modality=KnowledgeAgentOutputConfigurationModality.ANSWER_SYNTHESIS,
    include_activity=True,
)

agent = KnowledgeAgent(
    name=AGENT_NAME,
    models=[KnowledgeAgentAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(
            name=AGENT_NAME,
            reranker_threshold=2.5,
        )
    ],
    output_configuration=output_cfg,
)

index_client = SearchIndexClient(endpoint=AZURE_SEARCH_BASIC_ENDPOINT, credential=AzureKeyCredential(AZURE_SEARCH_BASIC_API_KEY))
index_client.create_or_update_agent(agent, api_version=AZURE_SEARCH_API_VERSION)
print(f"Knowledge agent '{AGENT_NAME}' created or updated successfully.")

## **SET UP MESSAGES**

In [ ]:
instructions = """
A Q&A agent that can answer questions about the Earth at night.
If you don't have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

## **RUN THE RETRIEVAL PIPELINE**

In [ ]:
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, SearchIndexKnowledgeSourceParams

agent_client = KnowledgeAgentRetrievalClient(
    endpoint=AZURE_SEARCH_BASIC_ENDPOINT, 
    agent_name=AGENT_NAME, 
    credential=AzureKeyCredential(AZURE_SEARCH_BASIC_API_KEY)
    )

query_1 = input("Enter your question about Earth at night: ") 
# e.g., "Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?"


messages.append({
    "role": "user",
    "content": query_1
})

req = KnowledgeAgentRetrievalRequest(
    messages=[
        KnowledgeAgentMessage(
            role=m["role"],
            content=[KnowledgeAgentMessageTextContent(text=m["content"])]
        ) for m in messages if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=KNOWLEDGE_SOURCE_NAME,
            kind="searchIndex"
        )
    ]
)

result = agent_client.retrieve(retrieval_request=req, api_version=AZURE_SEARCH_API_VERSION)
print(f"Retrieved content from '{KNOWLEDGE_SOURCE_NAME}' successfully.")

## **DELETING SOURCES**

In [ ]:
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_BASIC_ENDPOINT, credential=AzureKeyCredential(AZURE_SEARCH_BASIC_API_KEY))
index_client.delete_agent(AGENT_NAME)
print(f"Knowledge agent '{AGENT_NAME}' deleted successfully.")


index_client = SearchIndexClient(endpoint=AZURE_SEARCH_BASIC_ENDPOINT, credential=AzureKeyCredential(AZURE_SEARCH_BASIC_API_KEY))
index_client.delete_knowledge_source(knowledge_source=KNOWLEDGE_SOURCE_NAME)
print(f"Knowledge source '{KNOWLEDGE_SOURCE_NAME}' deleted successfully.")


index_client = SearchIndexClient(endpoint=AZURE_SEARCH_BASIC_ENDPOINT, credential=AzureKeyCredential(AZURE_SEARCH_BASIC_API_KEY))
index_client.delete_index(INDEX_NAME)
print(f"Index '{INDEX_NAME}' deleted successfully.")